# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_adams import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
my_weather = "output_data/my_weather_data.csv"

vacation_df = pd.read_csv(my_weather)

vacation_df


,City,Country,Date,Lattitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,Bredasdorp,ZA,1644118454,-34.5322,20.0403,64,95,20.16,2.36
1,Constitución,CL,1644118736,-35.3333,-72.4167,0,64,16.60,3.67
2,Albany,US,1644118738,42.6001,-73.9662,5,76,-12.03,2.02
3,Freeport,US,1644118457,40.6576,-73.5832,0,63,-5.55,0.89
4,Hilo,US,1644118740,19.7297,-155.0900,20,77,24.85,3.09
...,...,...,...,...,...,...,...,...,...
546,San Miguel de Tucumán,AR,1644119590,-26.8241,-65.2226,100,88,18.00,2.57
547,Port Macquarie,AU,1644119309,-31.4333,152.9167,74,88,19.98,4.63
548,Maragogi,BR,1644119102,-9.0122,-35.2225,81,77,25.07,2.77
549,Yaan,NG,1644119594,7.3833,8.5667,0,19,23.30,3.02


In [3]:
vacation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             551 non-null    object 
 1   Country          546 non-null    object 
 2   Date             551 non-null    int64  
 3   Lattitude        551 non-null    float64
 4   Longitude        551 non-null    float64
 5   Cloudiness       551 non-null    int64  
 6   Humidity         551 non-null    int64  
 7   Max Temperature  551 non-null    float64
 8   Wind Speed       551 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 38.9+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [9]:
locations = vacation_df.loc[:, ["Lattitude", "Longitude"]]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=vacation_df['Humidity'],max_intensity=300, point_radius=5, dissipating= False)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#Data is in Celcius. Converted to personal preferences in Celcius.
mask= (vacation_df["Max Temperature"] < 32) & (vacation_df["Max Temperature"] > 18) & (vacation_df["Wind Speed"] < 10) & (vacation_df.Cloudiness == 0)
my_cities= vacation_df.loc[mask]
my_cities

,City,Country,Date,Lattitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
13,Vallenar,CL,1644118753,-28.5708,-70.7581,0,54,21.26,0.66
21,Cape Town,ZA,1644118739,-33.9258,18.4232,0,73,21.73,0.45
24,Vila Velha,BR,1644118770,-20.3297,-40.2925,0,85,24.97,0.45
29,Kapaa,US,1644118778,22.0752,-159.3190,0,73,25.99,0.89
36,Kahului,US,1644118787,20.8947,-156.4700,0,71,26.01,7.20
78,New Norfolk,AU,1644118860,-42.7826,147.0587,0,34,24.81,1.34
127,Valparaíso,CL,1644118938,-33.0393,-71.6273,0,79,19.71,3.60
129,Praia,CV,1644118941,14.9215,-23.5087,0,78,21.30,6.69
138,Te Anau,NZ,1644118953,-45.4167,167.7167,0,39,18.78,2.53
175,Charters Towers,AU,1644118956,-20.1000,146.2667,0,34,30.68,4.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_names = []
ratings = []
cities = []
countries = []
lats = []
longs = []


hotel_df= my_cities[["City", "Country", "Lattitude", "Longitude"]]

hotel_df["hotel name"]= ""

for index, row in hotel_df.iterrows():
    print(row.City)
    
    target_coordinates = f"{row.Lattitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    places_data = response.json()

    try:
        hotel_df.loc[index, "hotel name"] = places_data["results"][0]["name"]
    except:
        print("Hotel not found")
hotel_df

/var/folders/5f/xys93hxj6gb7ymsc8c000z5h0000gn/T/ipykernel_7138/2129552710.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"]= ""


Vallenar


/Users/serendipitousfun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Cape Town
Vila Velha
Kapaa
Kahului
New Norfolk
Valparaíso
Praia
Te Anau
Charters Towers
East London
São Filipe
Ovalle
Praia
Pisco
Yulara
Formosa
Lázaro Cárdenas
Yaan
Hotel note found


,City,Country,Lattitude,Longitude,hotel name
13,Vallenar,CL,-28.5708,-70.7581,Humacao Bed & Breakfast
21,Cape Town,ZA,-33.9258,18.4232,Southern Sun Waterfront Cape Town
24,Vila Velha,BR,-20.3297,-40.2925,Quality Suites Vila Velha
29,Kapaa,US,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Kahului,US,20.8947,-156.4700,Maui Seaside Hotel
78,New Norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
127,Valparaíso,CL,-33.0393,-71.6273,Hotel Ultramar
129,Praia,CV,14.9215,-23.5087,LT Aparthotel
138,Te Anau,NZ,-45.4167,167.7167,Kingsgate Hotel Te Anau
175,Charters Towers,AU,-20.1000,146.2667,Cattleman's Rest Motor Inn


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#heatmap_layer = gmaps.heatmap_layer(locations, weights=hotel_df['Rating'],max_intensity=20, point_radius=5)

fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
#fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))